# Анализ рынка вакансий для аналитиков данных в Европе

**Цель работы**:

Цель данной работы - проанализировать и визуализировать информацию о рынке вакансий для аналитиков данных в Европейском регионе.


**Ход работы**:<a name="0"></a>

Данная работа пройдет в несколько этапов:

1. [**Загрузка и трансформация данных**](#1)

*Загрузка датасета и библиотек для работы, трансформация исходных сырых данных в датасет для дальнейшей обработки.*

2. [**Предобработка данных**](#2)

*Обработка пропусков, выбросов, явных и неявных дубликатов, а также проверка на соответствие ТЗ заказчика.*

3. [**Выгрузка и визуализация данных**](#3)

*Выгрузка итоговых данных и построение дашборда в Tableau.*


**Описание данных**:

Датасет содержит данные, полученные через парсинг сайта американской социальной сети для поиска и установления деловых контактов "LinkedIn". В исходном файле содержится список из 998 вакансий, связанных с ИТ-сферой и/или аналитикой данных в частности. Данные представляют собой отпечаток страницы с названием вакансии, географическом расположением, названием компании-нанимателя, сферой деятельности компании, типом занятости (офис, удаленная работа, гибрид, и т.п.), датой публикации вакансии относительно даты просмотра ("6 часов назад" ,"1 день назад", "2 недели назад", и т.п.), заявленное количество действующего персонала компании, число претендентов на вакансию, а также приводится основной текст вакансии, где в рамках данного исследования можно найти искомые компанией-нанимателем навыки работы с ПО (стек). 

<a name="1"></a>
## Первичный осмотр данных

[*вернуться в оглавление*](#0)

### Загрузка и трансформация данных

Так как данные предоставлены в виде отпечатков HTML-страниц, необходимо произвести трансформацию данных в формат, который можно будет использовать как для предобработки данных, так и для дальнейшей визуализации в дашборде. Для этого мы используем библиотеки `pandas` и `BeautifulSoup`, чтобы вытащить из сырых данных необходимые сведения и обьединить результат в единый датасет. 

Начнем работу с загрузки необходимых библиотек и исходных данных:

In [1]:
import os 
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
from IPython.display import display, HTML
import re
from warnings import filterwarnings
filterwarnings('ignore')
from tqdm.notebook import tqdm
tqdm.pandas()
import time
from time import sleep
import datetime
from datetime import date, datetime, timedelta

In [59]:
job_data = pd.read_csv('D:\Software\RawData\masterskaya_parsing_LinkedIn_2023_05_23.csv')

In [6]:
job_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 998 entries, 0 to 997
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  998 non-null    int64 
 1   html        998 non-null    object
dtypes: int64(1), object(1)
memory usage: 15.7+ KB


Данные представлены в виде датасета: первый столбец содержит номер страницы, а второй - ее содержание.

Выведем на экран одну из вакансий, чтобы определить где мы можем найти искомые данные в коде страниц:

In [7]:
display(HTML(job_data['html'][0]))

In [8]:
html = job_data['html'][0]

In [9]:
soup = BeautifulSoup(html)
soup

<html><body><div>
<div class="jobs-details__main-content jobs-details__main-content--single-pane full-width">
<!-- -->
<div>
<div class="jobs-unified-top-card t-14">
<!-- --> <div class="relative jobs-unified-top-card__container--two-pane">
<div class="jobs-unified-top-card__content--two-pane">
<!-- -->
<a class="ember-view" href="/jobs/view/3609065367/?alternateChannel=search&amp;refId=AxoCJAz0Vnn3xzCIlNO7ng%3D%3D&amp;trackingId=mdZWGb64G6DZP8qCAPQ%2BmQ%3D%3D&amp;trk=d_flagship3_search_srp_jobs" id="ember425">
<h2 class="t-24 t-bold jobs-unified-top-card__job-title">Data Analyst</h2>
</a>
<div class="jobs-unified-top-card__primary-description">
<span class="jobs-unified-top-card__subtitle-primary-grouping t-black">
<span class="jobs-unified-top-card__company-name">
<a class="ember-view t-black t-normal" href="/company/pharmiweb-jobs/life/" id="ember426">
                    PharmiWeb.Jobs: Global Life Science Jobs
                  </a>
</span>
<span class="jobs-unified-top-card__bull

Для большинства пунктов технического задания, необходимые сведения можно найти в соответствующих тегах и классах кода страницы - что мы и используем для добычи данных. Далее для этого мы вручную обьявим функции, которые вытащат нужный текст для отдельных тегов и классов. После проверки работоспособности функции, мы через цикл добавим к исходному датасету примененные функции как столбцы:

In [10]:
# находим название должности

soup.find('h2').text.strip()

def job_title(cell):
    "Функция вытаскивает из страницы название вакансии"
    soup = BeautifulSoup(cell)
    try:
        job_title = soup.find('h2').text.strip()
        return job_title
    except:
        return np.nan

In [11]:
# находим город, область (опционально) и страну вакансии

def job_city(cell):
    "Функция достает из страницы город, где предлагается вакансия"
    soup = BeautifulSoup(cell)
    try:
        job_location = (soup
                        .find('span', class_ = 'jobs-unified-top-card__bullet')
                        .text
                        .strip()
                        .split()
                       )
        return job_location[0]
    except:
        return np.nan

def job_country(cell):
    "Функция достает из страницы страну, где предлагается вакансия"
    soup = BeautifulSoup(cell)
    try:
        job_location = (soup
                        .find('span', class_ = 'jobs-unified-top-card__bullet')
                        .text
                        .strip()
                        .split()
                       )
        return job_location[-1]
    except:
        return np.nan

In [12]:
# находим тип занятости

def job_type(cell):
    "Функция достает из страницы тип занятости вакансии"
    soup = BeautifulSoup(cell)
    try:
        job_type = (soup
                    .find('span', class_ = 'jobs-unified-top-card__workplace-type')
                    .text
                    .strip()
                   )
        return job_type
    except:
        return np.nan

In [13]:
# находим название компании

def job_employer(cell):
    "Функция находит название компании-нанимателя"
    soup = BeautifulSoup(cell)
    try:
        job_employer = (soup
                        .find('a', class_ = 'ember-view t-black t-normal')
                        .text
                        .strip()
                       )
        return job_employer
    except:
        return np.nan

In [14]:
# находим количество работников

def staff_size(cell):
    "Функция находит заявленное число работников компании (размер штата)"
    soup = BeautifulSoup(cell)
    try:
        job_staff = (soup
                     .find('div', class_ = 't-14 mt5')
                     .text
                     .strip()
                     .split()[:-4][-1]
                    )
        return job_staff
    except:
        return np.nan

In [15]:
# находим сферу деятельности 

def field_of_work(cell):
    "Функция находит в тегах вакансии сферу деятельности компании"
    soup = BeautifulSoup(cell)
    try:
        job_specialty = (soup
                         .find('div', class_ = 't-14 mt5')
                         .contents[0]
                         .strip()
                        )
        return job_specialty
    except:
        return np.nan

In [16]:
# находим необходимые хардскиллы

def skills_stack(cell):
    soup = BeautifulSoup(cell)
    skills = ([
    'a/b testing', 'ab testing', 'actian', 'adobe analytics', 'adobe audience manager',
    'adobe experience platform', 'adobe launch', 'adobe target', 'ai', 'airflow',
    'alooma', 'alteryx', 'amazon machine learning', 'amazon web services', 'aml',
    'amplitude', 'ansible', 'apache camel', 'apache nifi', 'apache spark',
    'api', 'asana', 'auth0', 'aws', 'aws glue', 'azure', 'azure data factory',
    'basecamp', 'bash', 'beats', 'big query', 'bigquery', 'birst', 'bitbucket',
    'blendo', 'bootstrap', 'business objects bi', 'c#', 'c++', 'caffe', 'cassandra',
    'cdata sync', 'chronograf', 'ci/cd', 'cicd', 'clickhouse', 'cloudera', 'cluvio',
    'cntk', 'cognos', 'composer', 'computer vision', 'conda', 'confluence',
    'couchbase', 'css', 'd3.js', 'dash', 'dashboard', 'data factory', 'data fusion',
    'data mining', 'data studio', 'data warehouse', 'databricks', 'dataddo',
    'dataflow', 'datahub', 'dataiku', 'datastage', 'dbconvert', 'dbeaver', 'dbt',
    'deep learning', 'dl/ml', 'docker', 'domo', 'dune', 'dv360', 'dynamodb',
    'elasticsearch', 'elt', 'erwin', 'etl', 'etleap', 'excel', 'facebook business manager',
    'fivetran', 'fuzzy', 'ga360', 'gcp', 'gensim', 'ggplot', 'git', 'github', 'gitlab',
    'google ads', 'google analytics', 'google cloud platform', 'google data flow',
    'google optimize', 'google sheets', 'google tag manager', 'google workspace',
    'grafana', 'hadoop', 'hana', 'hanagrafana', 'hbase', 'hdfs', 'hevo data', 'hightouch',
    'hive', 'hivedatabricks', 'html', 'hubspot', 'ibm coremetrics', 'inetsoft',
    'influxdb', 'informatica', 'integrate.io', 'iri voracity', 'izenda', 'java',
    'java script', 'javascript', 'jenkins', 'jira', 'jmp', 'julia', 'jupyter',
    'k2view', 'kafka', 'kantar', 'kapacitor', 'keras', 'kibana', 'kubernetes',
    'lambda', 'linux', 'logstash', 'looker', 'lstm', 'luidgi', 'matillion', 'matlab',
    'matplotlib', 'mendix', 'metabase', 'microsoft sql', 'microsoft sql server',
    'microstrategy', 'miro', 'mixpanel', 'ml', 'ml flow', 'mlflow', 'mongodb', 'mxnet',
    'mysql', 'natural nanguage processing', 'neo4j', 'nlp', 'nltk', 'nosql', 'numpy',
    'oauth', 'octave', 'omniture', 'omnituregitlab', 'openshift', 'openstack',
    'optimizely', 'oracle', 'oracle business intelligence', 'oracle data integrator',
    'pandas', 'panorama', 'pentaho', 'plotly', 'postgre', 'postgresql', 'posthog',
    'power amc', 'power bi', 'power point', 'powerbi', 'powerpivot', 'powerpoint',
    'powerquery', 'power query', 'pyspark', 'python', 'pytorch', 'pytorchhevo data', 'qlik',
    'qlik sense', 'qlikview', 'querysurge', 'r', 'raphtory', 'rapidminer', 'redash',
    'redis', 'redshift', 'retool', 'rivery', 'rust', 's3', 'sa360', 'salesforce', 'sap',
    'sap business objects', 'sas', 'sas visual analytics', 'scala', 'scikit-learn',
    'scipy', 'seaborn', 'segment', 'selenium', 'sem rush', 'semrush', 'shell', 'shiny',
    'singer', 'sisense', 'skyvia', 'snowflake', 'spacy', 'spark', 'sparkml', 'splunk',
    'spotfire', 'spreadsheet', 'spss', 'sql', 'ssis', 'sssr', 'stambia', 'statistics',
    'statsbot', 'stitch', 'streamlit', 'streamsets', 'svn', 't-sql', 'tableau', 'talend',
    'targit', 'tealium', 'telegraf', 'tensorflow', 'terraapi', 'terraform', 'theano',
    'thoughtspot', 'timeseries', 'trello', 'unix', 'vba', 'vtom', 'webfocus', 'wfh',
    'xplenty', 'xtract.io', 'yellowfin'])
    matched_skills_list=[]
    try:
        for i in skills:
            if i == 'c++':
                if re.search('\Wc\+\+\W', cell.lower()):
                    matched_skills_list.append(i)      
            # word_border + rewritten "i" in special symbols + word_border
            else:
                pattern = (
                r'(\b|\W)'
                + re.escape(i)
                + r'(\b|\W)'
                +'|'
                + r'(\b|\W)'
                +re.escape(i.replace(' ', ''))
                + r'(\b|\W)'
            )
            if re.search(pattern, cell.lower()):
                matched_skills_list.append(i)
        return matched_skills_list
    
    except:
        return np.nan

In [17]:
# находим время публикования вакансии

def date_of_post(cell):
    soup = BeautifulSoup(cell)
    posting_time = (soup
                    .find('span', class_ = 'jobs-unified-top-card__posted-date')
                    .text
                    .strip()
                    .split()
                   )
    end_date = date(2023, 5, 23)
    
    try:
        if posting_time[1] in ('months', 'month'):
            job_date = end_date - timedelta(days=(30 * int(posting_time[0])))
        if posting_time[1] in ('weeks', 'week'):
            job_date = end_date - timedelta(days=(7 * int(posting_time[0])))
        if posting_time[1] in ('days', 'day'):
            job_date = end_date - timedelta(days=int(posting_time[0]))
        if posting_time[1] in ('hours', 'hour'):
            job_date = end_date

        return job_date.isoformat()
    except:
        return np.nan

In [18]:
# находим количество кандидатов на вакансию

def applicants(cell):
    soup = BeautifulSoup(cell)
    try:
        job_applicants = int(soup
                             .find('span', class_ = 'jobs-unified-top-card__applicant-count')
                             .text
                             .strip()
                             .split()[0]
                            )
        return job_applicants
    except:
        return np.nan

Все основные функции составлены, посмотрим их в работе:

In [19]:
job_data

,Unnamed: 0,html
0,0,"\n <div>\n <div class=""\n jobs-deta..."
1,1,"\n <div>\n <div class=""\n jobs-deta..."
2,2,"\n <div>\n <div class=""\n jobs-deta..."
3,3,"\n <div>\n <div class=""\n jobs-deta..."
4,4,"\n <div>\n <div class=""\n jobs-deta..."
...,...,...
993,993,"\n <div>\n <div class=""\n jobs-deta..."
994,994,"\n <div>\n <div class=""\n jobs-deta..."
995,995,"\n <div>\n <div class=""\n jobs-deta..."
996,996,"\n <div>\n <div class=""\n jobs-deta..."


In [20]:
func_list = [job_title,
             job_city,
             job_country,
             job_type,
             job_employer,
             staff_size,
             field_of_work,
             skills_stack,
             date_of_post,
             applicants]

for i in range(len(func_list)):
    job_data[func_list[i].__name__] = job_data['html'].progress_apply(func_list[i])

  0%|          | 0/998 [00:00<?, ?it/s]

  0%|          | 0/998 [00:00<?, ?it/s]

  0%|          | 0/998 [00:00<?, ?it/s]

  0%|          | 0/998 [00:00<?, ?it/s]

  0%|          | 0/998 [00:00<?, ?it/s]

  0%|          | 0/998 [00:00<?, ?it/s]

  0%|          | 0/998 [00:00<?, ?it/s]

  0%|          | 0/998 [00:00<?, ?it/s]

  0%|          | 0/998 [00:00<?, ?it/s]

  0%|          | 0/998 [00:00<?, ?it/s]

In [21]:
job_data

,Unnamed: 0,html,job_title,job_city,job_country,job_type,job_employer,staff_size,field_of_work,skills_stack,date_of_post,applicants
0,0,"\n <div>\n <div class=""\n jobs-deta...",Data Analyst,"Basel,",Switzerland,On-site,PharmiWeb.Jobs: Global Life Science Jobs,11-50,Staffing & Recruiting,"[data mining, excel, sap, sas, spss, sql, stat...",2023-05-16,47.0
1,1,"\n <div>\n <div class=""\n jobs-deta...",Data Analyst - Logistics,"Coventry,",Kingdom,On-site,NaN,NaN,NaN,[],2023-05-16,NaN
2,2,"\n <div>\n <div class=""\n jobs-deta...",Data Analyst - Logistics,"Coventry,",Kingdom,On-site,NaN,NaN,NaN,[wfh],2023-05-16,NaN
3,3,"\n <div>\n <div class=""\n jobs-deta...",Data Analyst (Space & Planning),South,Kingdom,On-site,NaN,NaN,NaN,[excel],2023-05-16,NaN
4,4,"\n <div>\n <div class=""\n jobs-deta...",Data Analyst,"Lugano,",Switzerland,On-site,NaN,NaN,NaN,"[aws, data warehouse, etl, gcp, oracle, oracle...",2023-05-09,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
993,993,"\n <div>\n <div class=""\n jobs-deta...",Ingeniero de datos Power BI + Azure Datafactory,Spain,Spain,Remote,Apiux Tecnología,201-500,Information Technology & Services,"[api, azure, data factory, power bi]",2023-05-19,55.0
994,994,"\n <div>\n <div class=""\n jobs-deta...",Stage | Data Analyst,"Vermezzo,",Italy,Hybrid,CPM Italy,51-200,Retail,"[ai, excel, power point, powerpoint]",2023-05-09,NaN
995,995,"\n <div>\n <div class=""\n jobs-deta...",Business-Analyst:in,Greater,Area,Hybrid,Computer Futures,"501-1,000",Information Technology & Services,"[excel, sql]",2023-05-22,51.0
996,996,"\n <div>\n <div class=""\n jobs-deta...",Junior Software Developer,"Milan,",Italy,Hybrid,XCHANGING ITALY S.P.A.,201-500,Computer Software,"[docker, java, nosql, sql]",2023-05-19,125.0


### Выводы - Первичный осмотр данных

- Поиск названия предлагаемой должности через обьявленную функцию не вызвал проблем, но сами должности из-за отсутствия более явной стандартизации при составлении потребуют предобработки - иначе получается слишком большое количество уникальных названий, что затуманивает представление о положении дел.


- Через работу функций удалось выделить большинство городов и стран предлагаемых вакансий. Однако если с городами неполные названия (в большинстве случаев - артикли "La", "Le", "The" или "Area" как указание пригородной зоны) представляют единичные случаи, то в случае стран точно заметны ошибки, которые связаны с особенностями заполнения географических данных самим сайтом "LinkedIn" ("United Kingdom" записано как "Kingdom", "Hague" является провинцией/метрополией Нидерландов, также встречается "Area" вместо страны). В ходе предобработки данных необходимо рассмотреть подробнее долю аномальных значений.


- Тип занятости для большинства вакансий был найден, однако в списке присутствуют пропущенные значения. Сами данные являются категориальными: `On-site` - работа в офисе/отведенной зоне, `Remote` - удаленная работа, `Hybrid` - гибридная работа. Так как функция по поиску типа занятости была написана вокруг унифицированного класса для этих значений, вероятно что причиной пропусков является отсутствие значений в теге на самой странице вакансии, а тип занятости указан в только теле обьявления. На этапе предобработки необходимо рассмотреть долю пропущенных значений.


- Названия компаний-нанимателей успешно помещены в список, однако были обнаружены пропущенные значения. Так как функция по поиску компаний построена вокруг унифицированного и общего для страниц данного типа тега и класса, возможной причиной возникновения пропусков является отсутствие данных в теге страниц, а само название компании-нанимателя указано только в теле обьявления. На этапе предобработки необходимо рассмотреть долю пропусков и возможную статистическую значимость.


- Размеры штатов компаний в данных вакансий указаны в виде категорий примерного числа сотрудников: `11-50`, `51-200`, `201-500`, `501-1000`, `1001-5000`, `5001-10000`, и `10001+`. В списке встречаются пропуски - вероятной причиной может быть отсутствие значений о размере штата сотрудников в общем для всех вакансий теге. На этапе предобработки необходимо рассмотреть долю пропусков для определения статистической значимости.


- Список сфер деятельности компаний датасета составлен, однако присутствуют пропуски в значениях. Так как функция по поиску данных построена вокруг общего для данных значений тега и класса, возможной причиной пропусков является отсутствие значений в теге вакансий. На этапе предобработки необходимо определить долю пропущенных значений для оценки статистической значимости.


- Функция по поиску стека вакансии отличается по строению от предыдущих функций. Так как для стека программ нет общего для всех вакансий тега и класса, функция использует сочетание регулярных выражений и поиск по ключевым словам из заранее составленного списка возможных программ. Из-за данных особенностей, выполнение нескольких циклов для заполнения списка занимает гораздо больше времени. В итоговом списке были обнаружены пропущенные значения - возможной причиной их возникновения является отсутствие ключевых слов в теле вакансии, несоответствие списка ключевых слов стеку вакансии из-за лингвистического или орфографического барьеров, или же отсутствие четко обозначенного стека в описании вакансии. На этапе предобработки необходимо подсчитать долю пропусков и оценить их статистическую значимость.


- Из-за особенностей исходных ориентиров по датам публикации, в списке встречается множество повторяющихся значений - скорее всего это обьясняется округлением значений в исходных данных. В списке дат публикаций вакансий также присутствуют единичные случаи пропусков. Возможной причиной возникновения является выход дат публикации данных вакансий за рамки временных периодов, указанных в обьявленной функции (т.е. более 11 месяцев с момента публикации). Так как данные пропуски встречаются в единичных случаях, их можно удалить без статистической значимости для датасета.


- В списке претендентов на вакансии обнаружено большое количество пропусков. Так как данные значения в теории появляются на сайте автоматически, возможной причиной их отсутствия является отсутствие претендентов на вакансию среди пользователей сайта. На этапе предобработки необходимо рассмотреть подробнее долю пропусков для возможного заполнения значением-заглушкой.

<a name="2"></a>
## Предобработка данных

[*вернуться в оглавление*](#0)

### Обзор данных датасета

Мы трансформировали данные в единый датафрейм `pandas` - далее проанализируем данные каждого столбца на предмет пропусков, дубликатов и других аномальных значений:

In [22]:
# job_data = pd.read_csv('D:\Software\RawData\job_data.csv')

In [23]:
job_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 998 entries, 0 to 997
Data columns (total 12 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Unnamed: 0     998 non-null    int64  
 1   html           998 non-null    object 
 2   job_title      998 non-null    object 
 3   job_city       998 non-null    object 
 4   job_country    998 non-null    object 
 5   job_type       930 non-null    object 
 6   job_employer   969 non-null    object 
 7   staff_size     961 non-null    object 
 8   field_of_work  964 non-null    object 
 9   skills_stack   998 non-null    object 
 10  date_of_post   996 non-null    object 
 11  applicants     838 non-null    float64
dtypes: float64(1), int64(1), object(10)
memory usage: 93.7+ KB


- Для фильтрации данных по ТЗ компании-заказчика (оставить начальные позиции для аналитиков данных, BI-аналитиков и Data Science) создадим новый столбец, где будет указано соответствие вакансии требованиям. Из-за особенностей исходных данных, далеко не у всех вакансий есть подходящие отметки в описании, хоть сами требования могут подходить. Само число таких вакансий составляет малую долю от данных, что не является желательным результатом. Поэтому, вместо того, чтобы оставить только начальные вакансии, мы исключим из датасета вакансии с указанием "мидл-сеньор-тимлид", где точно не требуются начинающие специалисты, и вакансии других сфер деятельности (маркетинг, инженеры данных, разработчики ПО, и т.п.).

In [24]:
def target_job(data):
        
        pattern_1 = r'\b\w*Anal|Scien|DS|DA\w*\b'
        pattern_2 = r'^(?!.*(?:teamlead|senior|mid)).*$'

        if (
            re.search(pattern_1, data, re.IGNORECASE) 
        and re.search(pattern_2, data, re.IGNORECASE)
        ):
            return 1
        else:
            return 0

job_data['target_job'] = job_data['job_title'].progress_apply(target_job)

job_data['target_job'].value_counts()

#job_data.query('target_job == 1')['job_title'].unique()

  0%|          | 0/998 [00:00<?, ?it/s]

1    852
0    146
Name: target_job, dtype: int64

In [25]:
job_data = job_data.query('target_job == 1')

In [26]:
job_data['job_title'] = job_data['job_title'].astype('string')
display(job_data['job_title'].describe())

count              852
unique             440
top       Data Analyst
freq                93
Name: job_title, dtype: object

In [27]:
job_data['job_city'] = job_data['job_city'].astype('string')

city = job_data['job_city']
city = city.str.replace(r'[^\w\s]', '')

job_data['job_city'] = city

display(job_data['job_city'].describe())

job_data['job_city'].value_counts()

count       852
unique      334
top       Milan
freq         32
Name: job_city, dtype: object

Milan        32
Paris        31
Prague       24
Frankfurt    24
Cologne      23
             ..
Bergamo       1
Grünheide     1
Leiden        1
Boadilla      1
Vermezzo      1
Name: job_city, Length: 334, dtype: Int64

- Для предобработки стран предлагаемых вакансий мы заменим обрезанное значение "United Kingdom" на "UK" и обрежем данные с неточным географическим положением (страна == "Area" или "Region"). Для предобработки городов предлагаемых вакансий уберем лишние знаки препинания, которые остались с этапа парсинга данных.

In [28]:
job_data['job_country'] = job_data['job_country'].replace('Kingdom', 'UK')
job_data = job_data.query('job_country != "Area" and job_country != "Region"')

job_data['job_country'] = job_data['job_country'].astype('string')
display(job_data['job_country'].describe())

job_data['job_country'].value_counts()

count         816
unique         30
top       Germany
freq          132
Name: job_country, dtype: object

Germany        132
Italy          124
France         104
UK              89
Netherlands     54
Spain           50
Poland          47
Belgium         35
Czechia         25
Hague           19
Portugal        19
Ireland         15
Sweden          14
Bulgaria        12
Greece          11
Luxembourg      11
Hungary         10
Romania          6
Lithuania        5
Austria          5
Denmark          4
Finland          4
Switzerland      4
Malta            4
Latvia           3
Croatia          3
Slovakia         3
Norway           2
Estonia          1
Monaco           1
Name: job_country, dtype: Int64

- Из-за особенностей исходных данных и формата составления не все вакансии имеют указание о типе занятости. Мы заполним пропуски значением "Not specified" (данные не указаны) как одной из имеющихся категорий. 

In [29]:
job_data.loc[:,('job_type')] = job_data.loc[:,('job_type')].fillna('Not specified')
job_data['job_type'] = job_data['job_type'].astype('category')
job_data['job_type'].describe()

count        816
unique         4
top       Hybrid
freq         379
Name: job_type, dtype: object

- Из-за особенностей исходных данных и формата составления не все вакансии имеют указанное название компании-нанимателя. Мы заполним пропуски значением "Unknown" (неизвестно). 

In [30]:
job_data.loc[:,('job_employer')] = job_data.loc[:,('job_employer')].fillna('Unknown')
job_data['job_employer'] = job_data['job_employer'].astype('string')
job_data['job_employer'].describe()

count           816
unique          504
top       CPM Italy
freq             45
Name: job_employer, dtype: object

- Из-за особенностей исходных данных и формата составления не все вакансии имеют указанный размер штата компании-нанимателя. Мы заполним пропуски значением "Not specified" (данные не указаны) как одной из имеющихся категорий. 

In [31]:
job_data.loc[:,('staff_size')] = job_data.loc[:,('staff_size')].fillna('Not specified')
job_data = job_data.query('staff_size != "Staffing" and staff_size != "&"')
job_data['staff_size'] = job_data['staff_size'].astype('category')
display(job_data['staff_size'].describe())

job_data['staff_size'].value_counts()

count         813
unique          9
top       10,001+
freq          232
Name: staff_size, dtype: object

10,001+          232
1,001-5,000      146
51-200           139
501-1,000         65
201-500           60
11-50             55
5,001-10,000      54
Not specified     35
2-10              27
Name: staff_size, dtype: int64

- Из-за особенностей исходных данных и формата составления не все вакансии имеют указанную сферу деятельности компании-нанимателя. Мы заполним пропуски значением "Unknown" (неизвестно). 

In [32]:
job_data.loc[:,('field_of_work')] = job_data.loc[:,('field_of_work')].fillna('Unknown')
job_data['field_of_work'] = job_data['field_of_work'].astype('string')
job_data['field_of_work'].describe()

count                                   813
unique                                   79
top       Information Technology & Services
freq                                    148
Name: field_of_work, dtype: object

- Среди указанных навыков претендентов было обнаружено большое количество пустых списков. Так как данные показатели не представляют ценности для дальнейшей работы, мы уберем их из датасета.

In [33]:
job_data = job_data.reset_index(drop=True)

job_data = job_data.query('skills_stack.notna() and skills_stack != "[]"')


job_data['skills_stack'].describe()

count     813
unique    456
top        []
freq       56
Name: skills_stack, dtype: object

In [34]:
job_data['date_of_post'] = pd.to_datetime(job_data['date_of_post'], format='%Y/%m/%d')

job_data = job_data.query('date_of_post.notna()')

job_data['date_of_post'].describe(datetime_is_numeric=True)

count                              811
mean     2023-05-12 13:33:13.094944256
min                2023-04-25 00:00:00
25%                2023-05-09 00:00:00
50%                2023-05-16 00:00:00
75%                2023-05-17 00:00:00
max                2023-05-23 00:00:00
Name: date_of_post, dtype: object

- Так как количество претендентов на вакансию заполняется автоматически со стороны сайта, отсутствие данных можно интерпретировать как отсутствие претендентов - поэтому мы заполним пропуски нулем.

In [35]:
job_data['applicants'] = job_data['applicants'].fillna(0)
job_data['applicants'] = job_data['applicants'].astype('int')
display(job_data['applicants'].describe())
job_data['applicants'].quantile([.9, .95, .99])

count    811.000000
mean      47.850801
std       48.369424
min        0.000000
25%        8.000000
50%       33.000000
75%       74.000000
max      198.000000
Name: applicants, dtype: float64

0.90    124.0
0.95    149.0
0.99    186.0
Name: applicants, dtype: float64

In [36]:
job_data = job_data.reset_index(drop=True)

In [37]:
job_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 811 entries, 0 to 810
Data columns (total 13 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   Unnamed: 0     811 non-null    int64         
 1   html           811 non-null    object        
 2   job_title      811 non-null    string        
 3   job_city       811 non-null    string        
 4   job_country    811 non-null    string        
 5   job_type       811 non-null    category      
 6   job_employer   811 non-null    string        
 7   staff_size     811 non-null    category      
 8   field_of_work  811 non-null    string        
 9   skills_stack   811 non-null    object        
 10  date_of_post   811 non-null    datetime64[ns]
 11  applicants     811 non-null    int32         
 12  target_job     811 non-null    int64         
dtypes: category(2), datetime64[ns](1), int32(1), int64(2), object(2), string(5)
memory usage: 68.8+ KB


In [45]:
job_data.head(10)

,Unnamed: 0,html,job_title,job_city,job_country,job_type,job_employer,staff_size,field_of_work,skills_stack,date_of_post,applicants,target_job
0,0,"\n <div>\n <div class=""\n jobs-deta...",Data Analyst,Basel,Switzerland,On-site,PharmiWeb.Jobs: Global Life Science Jobs,11-50,Staffing & Recruiting,"[data mining, excel, sap, sas, spss, sql, stat...",2023-05-16,47,1
1,1,"\n <div>\n <div class=""\n jobs-deta...",Data Analyst - Logistics,Coventry,UK,On-site,Unknown,Not specified,Unknown,[],2023-05-16,0,1
2,2,"\n <div>\n <div class=""\n jobs-deta...",Data Analyst - Logistics,Coventry,UK,On-site,Unknown,Not specified,Unknown,[wfh],2023-05-16,0,1
3,3,"\n <div>\n <div class=""\n jobs-deta...",Data Analyst (Space & Planning),South,UK,On-site,Unknown,Not specified,Unknown,[excel],2023-05-16,0,1
4,4,"\n <div>\n <div class=""\n jobs-deta...",Data Analyst,Lugano,Switzerland,On-site,Unknown,Not specified,Unknown,"[aws, data warehouse, etl, gcp, oracle, oracle...",2023-05-09,0,1
5,5,"\n <div>\n <div class=""\n jobs-deta...",Data Analyst - Logistics,Southampton,UK,On-site,Unknown,Not specified,Unknown,"[excel, power bi, powerbi, sap]",2023-05-17,0,1
6,6,"\n <div>\n <div class=""\n jobs-deta...",Data Analyst,Leeds,UK,On-site,Unknown,Not specified,Unknown,"[excel, power bi]",2023-05-02,0,1
7,7,"\n <div>\n <div class=""\n jobs-deta...",Data Analyst,Nuneaton,UK,Hybrid,Unknown,Not specified,Unknown,"[data mining, excel]",2023-05-21,0,1
8,8,"\n <div>\n <div class=""\n jobs-deta...",Data Analyst,Paris,France,On-site,eXalt,Not specified,Unknown,"[dataiku, nosql, power bi, powerbi, qlikview, ...",2023-05-09,140,1
9,9,"\n <div>\n <div class=""\n jobs-deta...",Data Analyst - Hybrid Working,Cambridge,UK,On-site,Unknown,Not specified,Unknown,[],2023-05-09,0,1


Дальнейшая визуализация пройдет в программе "Tableau" - для корректного подсчета данных в столбце `skills_stack` наиболее оптимальным решением будет выделить стек программ в отдельный датасет, вместе с названием вакансии и компании-нанимателя как ориентирами.

In [54]:
job_skills = (job_data[['job_title', 'job_employer', 'skills_stack']]
                    .reset_index()
                   )

job_skills.columns = ['job_id', 'job_title', 'job_employer', 'skills_stack']

job_skills['job_id'] += 1

# job_skills['skills_stack'] = (job_skills['skills_stack']
#                         .str.split(',')
#                        )

job_skills = (job_skills
              .set_index(['job_id'])
              .apply(pd.Series.explode)
              .reset_index()
             )

# punctuation = '''!()[]{};:'"\, <>./?@#$%^&*_~'''
# for x in punctuation:
#     job_skills['skills_stack'] = job_skills['skills_stack'].str.replace(x, "")

job_skills

,job_id,job_title,job_employer,skills_stack
0,1,Data Analyst,PharmiWeb.Jobs: Global Life Science Jobs,data mining
1,1,Data Analyst,PharmiWeb.Jobs: Global Life Science Jobs,excel
2,1,Data Analyst,PharmiWeb.Jobs: Global Life Science Jobs,sap
3,1,Data Analyst,PharmiWeb.Jobs: Global Life Science Jobs,sas
4,1,Data Analyst,PharmiWeb.Jobs: Global Life Science Jobs,spss
...,...,...,...,...
3615,810,Stage | Data Analyst,CPM Italy,ai
3616,810,Stage | Data Analyst,CPM Italy,excel
3617,810,Stage | Data Analyst,CPM Italy,power point
3618,810,Stage | Data Analyst,CPM Italy,powerpoint


### Выводы - Предобработка данных

- Для фильтрации данных по ТЗ компании-заказчика (оставить начальные позиции для аналитиков данных, BI-аналитиков и Data Science) создадим новый столбец, где будет указано соответствие вакансии требованиям. Из-за особенностей исходных данных, далеко не у всех вакансий есть подходящие отметки в описании, хоть сами требования могут подходить. Само число таких вакансий составляет малую долю от данных, что не является желательным результатом. Поэтому, вместо того, чтобы оставить только начальные вакансии, мы исключим из датасета вакансии с указанием "мидл-сеньор-тимлид", где точно не требуются начинающие специалисты, и вакансии других сфер деятельности (маркетинг, инженеры данных, разработчики ПО, и т.п.).


- Для предобработки стран предлагаемых вакансий мы заменим обрезанное значение "United Kingdom" на "UK" и обрежем данные с неточным географическим положением (страна == "Area" или "Region"). Для предобработки городов предлагаемых вакансий уберем лишние знаки препинания, которые остались с этапа парсинга данных.


- Из-за особенностей исходных данных и формата составления не все вакансии имеют указание о типе занятости. Мы заполним пропуски значением "Not specified" (данные не указаны) как одной из имеющихся категорий. 


- Из-за особенностей исходных данных и формата составления не все вакансии имеют указанное название компании-нанимателя. Мы заполним пропуски значением "Unknown" (неизвестно). 


- Из-за особенностей исходных данных и формата составления не все вакансии имеют указанный размер штата компании-нанимателя. Мы заполним пропуски значением "Not specified" (данные не указаны) как одной из имеющихся категорий. 


- Из-за особенностей исходных данных и формата составления не все вакансии имеют указанную сферу деятельности компании-нанимателя. Мы заполним пропуски значением "Unknown" (неизвестно). 


- Среди указанных навыков претендентов было обнаружено большое количество пустых списков. Так как данные показатели не представляют ценности для дальнейшей работы, мы уберем их из датасета.


- Так как количество претендентов на вакансию заполняется автоматически со стороны сайта, отсутствие данных можно интерпретировать как отсутствие претендентов - поэтому мы заполним пропуски нулем.

<a name="3"></a>
## Выгрузка и визуализация данных

[*вернуться в оглавление*](#0)

In [57]:
filtered_job_data = job_data.copy()
filtered_job_data.drop(columns='html').to_csv(r'D:\Software\RawData\filtered_job_data.csv')
job_skills.to_csv(r'D:\Software\RawData\job_skills.csv')

Итоговый дашборд доступен по ссылке: https://public.tableau.com/app/profile/mikhail.filimonov/viz/LinkedInEntry-LevelDataJobs/Dashboard1